In [1]:
import findspark
import os

findspark.init(spark_home='/Users/rhyando/spark')
os.environ['JAVA_HOME']='/opt/homebrew/Cellar/openjdk@11/11.0.23/libexec/openjdk.jdk/Contents/Home'

In [2]:
# test edit
# based on https://mybinder.org/v2/gh/projectnessie/nessie-demos/main?filepath=notebooks/nessie-iceberg-demo-nba.ipynb

import pyspark
from pyspark.sql import SparkSession
import os
import socket

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.82.0'

conf = pyspark.SparkConf().setAll([
    # ip of spark master
    # ('spark.driver.bindAddress', '10.244.0.14')
    ('spark.driver.host', socket.gethostbyname(socket.gethostname()))
    #, ('spark.ui.proxyBase', os.environ['JUPYTERHUB_SERVICE_PREFIX'] + 'proxy/4040')
    # ('spark.submit.deployMode', 'client')
    # , ('spark.driver.port', '7077')
    , ('spark.master', "spark://127.0.0.1:7077")
    # , ('spark.jars.packages', 'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.82.0,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-aws:3.3.4')
    , ("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
    # cluster ip
    # , ('spark.hadoop.fs.s3a.endpoint', 'http://10.104.71.148:6544')
    , ('spark.hadoop.fs.s3a.endpoint', "http://127.0.0.1:6544")
    # for minio spark
    , ('spark.hadoop.fs.s3a.access.key','minio')
    , ('spark.hadoop.fs.s3a.secret.key', 'minio123')
    , ('spark.hadoop.fs.s3a.path.style.access', 'true')
    , ("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # for iceberg minio jdbc
    #, ('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebesrgSparkSessionExtensions')
    , ('spark.sql.catalog.nessie_catalog', 'org.apache.iceberg.spark.SparkCatalog')
    , ("spark.sql.catalog.nessie_catalog.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    , ('spark.sql.defaultCatalog', 'nessie_catalog')
    , ('spark.sql.catalog.nessie_catalog.warehouse', 's3a://test/')
    # , ('spark.sql.catalog.my_catalog.type', 'jdbc')
    # ip of service postgre
    , ('spark.sql.catalog.nessie_catalog.uri', 'http://127.0.0.1:6788/api/v1')
    , ('spark.sql.catalog.nessie_catalog.ref', 'main')
    , ("spark.sql.catalog.nessie_catalog.authentication.type", 'NONE')
    # , ('spark.sql.catalog.my_catalog.jdbc.user', 'postgres')
    # , ('spark.sql.catalog.my_catalog.jdbc.password', 'postgres')
    ])
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/31 22:45:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("LIST REFERENCES IN nessie_catalog").show()

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch| dev|09ab65730d3ee184f...|
| Branch|main|2e1cfa82b035c26cb...|
+-------+----+--------------------+

